In [7]:
!pip install xmltodict

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pprint
import requests, json
import pandas as pd
import xmltodict
import pyodbc

In [2]:
def to_xml(df, filename=None, mode='w'):
    def row_to_xml(row):
        xml = ['<item>']
        for i, col_name in enumerate(row.index):
            xml.append('  <{0}>{1}</{0}>'.format(col_name, row.iloc[i]))
        xml.append('</item>')
        return '\n'.join(xml)
    res = '<items>'+'\n'.join(df.apply(row_to_xml, axis=1))+'</items>'

    if filename is None:
        return res
    with open(filename, mode) as f:
        f.write(res)

pd.DataFrame.to_xml = to_xml

In [3]:
Driver = '{SQL Server Native Client 11.0}'
SERVER = 'JUNZEZHANG-PC\SQLEXPRESS'
DATABASE = 'AdventureworksDW2012'
engine = 'DRIVER={0}; SERVER={1}; DATABASE={2}; Trusted_Connection=yes'.format(Driver, SERVER, DATABASE)
sql_conn = pyodbc.connect(engine) 

In [4]:
key = 'X1-ZWz1guz2g6to23_211k7'

# GetSearchResults

In [5]:
address = "1310 Poplar Pointe SE"
postal_code = "30082"
rentzestimate = True

In [6]:
url = "http://www.zillow.com/webservice/GetSearchResults.htm?zws-id={0}&address={1}&citystatezip={2}&rentzestimate={3}".format(key,address,postal_code, rentzestimate)
resp = requests.get(url)
xml_zillow = xmltodict.parse(resp.text)

In [7]:
xml_zillow

OrderedDict([('SearchResults:searchresults',
              OrderedDict([('@xsi:schemaLocation',
                            'http://www.zillow.com/static/xsd/SearchResults.xsd http://www.zillowstatic.com/vstatic/80d5e73/static/xsd/SearchResults.xsd'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('@xmlns:SearchResults',
                            'http://www.zillow.com/static/xsd/SearchResults.xsd'),
                           ('request',
                            OrderedDict([('address', '1310 Poplar Pointe SE'),
                                         ('citystatezip', '30082')])),
                           ('message',
                            OrderedDict([('text',
                                          'Request successfully processed'),
                                         ('code', '0')])),
                           ('response',
                            OrderedDict([('

In [8]:
dict_zillow = dict()

In [9]:
dict_zillow['zpid'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['zpid']
dict_zillow['homedetails'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['links']['homedetails']
dict_zillow['graphsanddata'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['links']['graphsanddata']
dict_zillow['mapthishome'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['links']['mapthishome']
dict_zillow['comparables'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['links']['comparables']
dict_zillow['address_street'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['address']['street']
dict_zillow['address_zipcode'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['address']['zipcode']
dict_zillow['address_city'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['address']['city']
dict_zillow['address_state'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['address']['state']
dict_zillow['address_latitude'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['address']['latitude']
dict_zillow['address_longitude'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['address']['longitude']
dict_zillow['zestimate_amount'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['zestimate']['amount']['#text']
dict_zillow['zestimate_amount_low'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['zestimate']['valuationRange']['low']['#text']
dict_zillow['zestimate_amount_high'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['zestimate']['valuationRange']['high']['#text']
dict_zillow['zestimate_lastupdated'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['zestimate']['last-updated']
dict_zillow['rentzestimate_amount'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['rentzestimate']['amount']['#text']
dict_zillow['rentzestimate_amount_low'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['rentzestimate']['valuationRange']['low']['#text']
dict_zillow['rentzestimate_amount_high'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['rentzestimate']['valuationRange']['high']['#text']
dict_zillow['rentzestimate_lastupdated'] = xml_zillow['SearchResults:searchresults']['response']['results']['result']['rentzestimate']['last-updated']

In [10]:
df_zillow = pd.DataFrame(dict_zillow, index = range(1))
df_zillow

,zpid,homedetails,graphsanddata,mapthishome,comparables,address_street,address_zipcode,address_city,address_state,address_latitude,address_longitude,zestimate_amount,zestimate_amount_low,zestimate_amount_high,zestimate_lastupdated,rentzestimate_amount,rentzestimate_amount_low,rentzestimate_amount_high,rentzestimate_lastupdated
0,14297694,http://www.zillow.com/homedetails/1310-Poplar-...,http://www.zillow.com/homedetails/1310-Poplar-...,http://www.zillow.com/homes/14297694_zpid/,http://www.zillow.com/homes/comps/14297694_zpid/,1310 Poplar Pointe SE,30082,Smyrna,GA,33.880968,-84.541846,151891,141259,161004,09/12/2019,1370,1260,1562,09/12/2019


In [30]:
df_zillow_xml = df_zillow.to_xml()

In [33]:
print(df_zillow_xml)

<items><item>
  <zpid>14299307</zpid>
  <homedetails>http://www.zillow.com/homedetails/2166-Trailwood-Dr-SE-Smyrna-GA-30080/14299307_zpid/</homedetails>
  <graphsanddata>http://www.zillow.com/homedetails/2166-Trailwood-Dr-SE-Smyrna-GA-30080/14299307_zpid/#charts-and-data</graphsanddata>
  <mapthishome>http://www.zillow.com/homes/14299307_zpid/</mapthishome>
  <comparables>http://www.zillow.com/homes/comps/14299307_zpid/</comparables>
  <address_street>2166 Trailwood Dr SE</address_street>
  <address_zipcode>30080</address_zipcode>
  <address_city>Smyrna</address_city>
  <address_state>GA</address_state>
  <address_latitude>33.900247</address_latitude>
  <address_longitude>-84.536922</address_longitude>
  <zestimate_amount>177294</zestimate_amount>
  <zestimate_amount_low>152473</zestimate_amount_low>
  <zestimate_amount_high>209207</zestimate_amount_high>
  <zestimate_lastupdated>09/10/2019</zestimate_lastupdated>
  <rentzestimate_amount>1350</rentzestimate_amount>
  <rentzestimate_amo

In [34]:
try:
    connstr = pyodbc.connect(engine) 
    cursor = connstr.cursor()

    cursor.execute("exec sp_insertGetZillowSearchResults @XML = '{0}'".format(df_zillow_xml))

    connstr.commit()
    cursor.close()
    connstr.close()
    print('success')
except:
    print('fail')

success


# GetComps

In [172]:
zpid = '14314229'
count=2
rentzestimate = True

In [173]:
url = 'http://www.zillow.com/webservice/GetComps.htm?zws-id={0}&zpid={1}&count={2}&rentzestimate={3}'.format(key, zpid, count,rentzestimate)
resp_comps = requests.get(url)
xml_getcomp = xmltodict.parse(resp_comps.text)

In [174]:
dict_getcomp = dict()

In [175]:
dict_getcomp['zpid_principle'] = xml_getcomp['Comps:comps']['response']['properties']['principal']['zpid']
dict_getcomp['zpid_comparables'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['zpid'] for n in range(count)]
dict_getcomp['homedetails'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['links']['homedetails'] for n in range(count)]
dict_getcomp['graphsanddata'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['links']['graphsanddata'] for n in range(count)]
dict_getcomp['mapthishome'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['links']['mapthishome'] for n in range(count)]
dict_getcomp['comparables'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['links']['comparables'] for n in range(count)]
dict_getcomp['address_street'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['address']['street'] for n in range(count)]
dict_getcomp['address_zipcode'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['address']['zipcode'] for n in range(count)]
dict_getcomp['address_city'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['address']['city'] for n in range(count)]
dict_getcomp['address_state'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['address']['state'] for n in range(count)]
dict_getcomp['address_latitude'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['address']['latitude'] for n in range(count)]
dict_getcomp['address_longitude'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['address']['longitude'] for n in range(count)]
dict_getcomp['zestimate_amount'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['zestimate']['amount']['#text'] for n in range(count)]
dict_getcomp['zestimate_amount_low'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['zestimate']['valuationRange']['low']['#text'] for n in range(count)]
dict_getcomp['zestimate_amount_high'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['zestimate']['valuationRange']['high']['#text'] for n in range(count)]
dict_getcomp['zestimate_lastupdated'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['zestimate']['last-updated'] for n in range(count)]
dict_getcomp['rentzestimate_amount'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['rentzestimate']['amount']['#text'] for n in range(count)]
dict_getcomp['rentzestimate_amount_low'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['rentzestimate']['valuationRange']['low']['#text'] for n in range(count)]
dict_getcomp['rentzestimate_amount_high'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['rentzestimate']['valuationRange']['high']['#text'] for n in range(count)]
dict_getcomp['rentzestimate_lastupdated'] = [xml_getcomp['Comps:comps']['response']['properties']['comparables']['comp'][n]['rentzestimate']['last-updated'] for n in range(count)]

In [154]:
getcomp_zillow = pd.DataFrame(dict_getcomp, index = range(count))
getcomp_zillow.head(5)

,zpid_principle,zpid_comparables,homedetails,graphsanddata,mapthishome,comparables,address_street,address_zipcode,address_city,address_state,address_latitude,address_longitude,zestimate_amount,zestimate_amount_low,zestimate_amount_high,zestimate_lastupdated,rentzestimate_amount,rentzestimate_amount_low,rentzestimate_amount_high,rentzestimate_lastupdated
0,14314229,14312615,https://www.zillow.com/homedetails/2728-Farmst...,http://www.zillow.com/homedetails/2728-Farmste...,http://www.zillow.com/homes/14312615_zpid/,http://www.zillow.com/homes/comps/14312615_zpid/,2728 Farmstead Rd SE,30080,SMYRNA,GA,33.885932,-84.485396,174277,165563,182991,02/08/2019,1370,1206,1493,02/03/2019
1,14314229,14312577,https://www.zillow.com/homedetails/2792-Farmst...,http://www.zillow.com/homedetails/2792-Farmste...,http://www.zillow.com/homes/14312577_zpid/,http://www.zillow.com/homes/comps/14312577_zpid/,2792 Farmstead Rd SE,30080,SMYRNA,GA,33.884987,-84.4849,161468,153395,169541,02/08/2019,1275,1096,1441,02/03/2019
2,14314229,14312714,https://www.zillow.com/homedetails/2580-Rollin...,http://www.zillow.com/homedetails/2580-Rolling...,http://www.zillow.com/homes/14312714_zpid/,http://www.zillow.com/homes/comps/14312714_zpid/,2580 Rolling View Dr SE,30080,SMYRNA,GA,33.889687,-84.485412,237731,225844,249618,02/08/2019,1420,1349,1633,02/03/2019
3,14314229,14312726,https://www.zillow.com/homedetails/2698-Rollin...,http://www.zillow.com/homedetails/2698-Rolling...,http://www.zillow.com/homes/14312726_zpid/,http://www.zillow.com/homes/comps/14312726_zpid/,2698 Rolling View Dr SE,30080,SMYRNA,GA,33.886895,-84.485469,238098,226193,250003,02/08/2019,1450,1378,1639,02/03/2019
4,14314229,14312816,https://www.zillow.com/homedetails/2476-Caroly...,http://www.zillow.com/homedetails/2476-Carolyn...,http://www.zillow.com/homes/14312816_zpid/,http://www.zillow.com/homes/comps/14312816_zpid/,2476 Carolyn Dr SE,30080,SMYRNA,GA,33.892169,-84.48656,281551,267473,295629,02/08/2019,1400,1302,1610,02/03/2019


In [155]:
getcomp_zillow_xml = getcomp_zillow.to_xml()

In [163]:
try:
    connstr = pyodbc.connect(engine) 
    cursor = connstr.cursor()

    cursor.execute("exec sp_insertGetZillowComps @XML = '{0}'".format(getcomp_zillow_xml))

    connstr.commit()
    cursor.close()
    connstr.close()
    print('success')
except:
    print('fail')

success
